<a href="https://colab.research.google.com/github/uribo1008/B5_TextProcessing2/blob/main/B6_NaiveBayesClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 単純ベイズ分類器

### パッケージの読み込み

ここではガウス分布の単純ベイズ分類器（GaussianNB）を利用。単純ベイズ分類器を含む機械学習パッケージ（scikit-learn）については資料を参照のこと。

In [1]:
import numpy as np
from sklearn.naive_bayes import GaussianNB

### 学習セットの用意

学習セットの説明については資料を参照のこと。

In [2]:
X = np.array([[1, 0, 1, 1, 2, 1, 1, 0, 0],
              [1, 2, 0, 1, 0, 0, 3, 3, 1],
              [2, 0, 1, 0, 3, 1, 0, 2, 1]]) #特徴ベクトル　数値は適当
y = np.array([1, 2, 3])#クラス分け

### 単純ベイズ分類器の学習

In [3]:
clf = GaussianNB()

In [4]:
clf.fit(X, y)

GaussianNB()

In [5]:
clf.score(X, y)

1.0

### 学習済の分類器を用いた推定

In [6]:
t = np.array([[1, 0, 1, 1, 0, 0, 1, 0, 0]])
clf.predict(t)

array([1])

# 2. テキストの分類

### パッケージの読み込み

CountVectorizerは単語の数を数えるパッケージで，これを利用してBoWベクトルを作成します。

In [7]:
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import CountVectorizer

### 学習セットの用意

In [8]:
s = ['今日 は とても 天気 が いい',
     '今日 は 晴れ です',
     '天気 が いい 日 は 晴れ です']

In [9]:
vectorizer = CountVectorizer()
vectorizer.fit(s)
X = vectorizer.transform(s).toarray()

In [16]:
X #これは６次元のベクトル

array([[1, 0, 1, 1, 1, 0],
       [0, 1, 0, 1, 0, 1],
       [1, 1, 0, 0, 1, 1]])

In [17]:
y = np.array([1, 2, 3])

### 単純ベイズ分類器の学習

In [18]:
clf = GaussianNB()

In [19]:
clf.fit(X, y)

GaussianNB()

In [20]:
clf.score(X, y) #1.0はうまくいっている

1.0

### 学習済分類器を用いた推定

In [21]:
t = vectorizer.transform(["明日 は 天気 が いい"]).toarray()
clf.predict(t)

array([1])

# 3. 長いテキストの分類

### janome（形態素解析）のインストール

In [22]:
!pip install janome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 70.6 MB/s eta 0:00:00


### パッケージの読み込み

In [23]:
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from janome.tokenizer import Tokenizer

### 学習用文書の読み込み
まず，テキストファイルをアップロードします。
続いて以下のセルを実行し，15つアップしたうち，12を学習用に，3つはテスト用にします。

In [24]:
# テキスト読み込み用の関数定義
def file_load(file_name):
  text = None
  with open(file_name, "r") as f:
    text = "\n".join(f.readlines())
  return text

In [26]:
file_list = [
    # 夏目漱石
    "bot_content.txt",
    "34ro_content.txt",
    "kokoro_content.txt",
    "sore_content.txt",
    "meian_content.txt",
    # 芥川竜之介
    "jigokuhen_content.txt",
    "kappa_content.txt",
    "syuju_content.txt",
    # 太宰治
    "ningen_content.txt",
    "shayou_content.txt",
    "hamlet_content.txt",
    "pandora_content.txt"
]
text_list = []
for file in file_list:
  text_list.append(file_load(file))

In [27]:
# 最初のテキスト（坊ちゃん）の最初の1000文字を表示
text_list[0][0:1000]

'<br/>\n\n<div class="jisage_5" style="margin-left: 5em"><h4 class="naka-midashi"><a class="midashi_anchor" id="midashi10">一</a></h4></div>\n\n<br/>\n\n\u3000<ruby><rb>親譲</rb><rp>（</rp><rt>おやゆず</rt><rp>）</rp></ruby>りの<ruby><rb>無鉄砲</rb><rp>（</rp><rt>むてっぽう</rt><rp>）</rp></ruby>で小供の時から損ばかりしている。小学校に居る時分学校の二階から飛び降りて一週間ほど<ruby><rb>腰</rb><rp>（</rp><rt>こし</rt><rp>）</rp></ruby>を<ruby><rb>抜</rb><rp>（</rp><rt>ぬ</rt><rp>）</rp></ruby>かした事がある。なぜそんな<ruby><rb>無闇</rb><rp>（</rp><rt>むやみ</rt><rp>）</rp></ruby>をしたと聞く人があるかも知れぬ。別段深い理由でもない。新築の二階から首を出していたら、同級生の一人が<ruby><rb>冗談</rb><rp>（</rp><rt>じょうだん</rt><rp>）</rp></ruby>に、いくら<ruby><rb>威張</rb><rp>（</rp><rt>いば</rt><rp>）</rp></ruby>っても、そこから飛び降りる事は出来まい。弱虫やーい。と<ruby><rb>囃</rb><rp>（</rp><rt>はや</rt><rp>）</rp></ruby>したからである。<ruby><rb>小使</rb><rp>（</rp><rt>こづかい</rt><rp>）</rp></ruby>に負ぶさって帰って来た時、おやじが大きな<ruby><rb>眼</rb><rp>（</rp><rt>め</rt><rp>）</rp></ruby>をして二階ぐらいから飛び降りて腰を抜かす<ruby><rb>奴</rb><rp>（</rp><rt>やつ</rt><rp>）</rp></ruby>があるかと<ruby><rb>云</rb><rp>（</rp><rt>い</rt><rp>

### クレンジング

まずクレンジング用の関数を定義して，そのあとクレンジング

In [28]:
import re

def cleansing(text):
  clean_text = re.sub("\s", "", text)                     # 余分な空白（改行や字下げの空白）を除去
  clean_text = clean_text.replace("<br/>", "\n")          # <br/>タグを改行に変換
  clean_text = re.sub(r"<rp>[^<]+</rp>", "", clean_text)  # ルビの前後の括弧を除去
  clean_text = re.sub(r"<rt>[^<]+</rt>", "", clean_text)  # ルビのテキストを除去
  clean_text = re.sub(r"<[^>]+>", "", clean_text)         # それ以外のタグを除去
  return clean_text

In [29]:
clean_text_list = []
for text in text_list:
  clean_text_list.append(cleansing(text))

In [30]:
# 最初のテキストの最初の1000文字を表示
clean_text_list[0][0:1000]

'\n一\n親譲りの無鉄砲で小供の時から損ばかりしている。小学校に居る時分学校の二階から飛び降りて一週間ほど腰を抜かした事がある。なぜそんな無闇をしたと聞く人があるかも知れぬ。別段深い理由でもない。新築の二階から首を出していたら、同級生の一人が冗談に、いくら威張っても、そこから飛び降りる事は出来まい。弱虫やーい。と囃したからである。小使に負ぶさって帰って来た時、おやじが大きな眼をして二階ぐらいから飛び降りて腰を抜かす奴があるかと云ったから、この次は抜かさずに飛んで見せますと答えた。\n親類のものから西洋製のナイフを貰って奇麗な刃を日に翳して、友達に見せていたら、一人が光る事は光るが切れそうもないと云った。切れぬ事があるか、何でも切ってみせると受け合った。そんなら君の指を切ってみろと注文したから、何だ指ぐらいこの通りだと右の手の親指の甲をはすに切り込んだ。幸ナイフが小さいのと、親指の骨が堅かったので、今だに親指は手に付いている。しかし創痕は死ぬまで消えぬ。\n庭を東へ二十歩に行き尽すと、南上がりにいささかばかりの菜園があって、真中に栗の木が一本立っている。これは命より大事な栗だ。実の熟する時分は起き抜けに背戸を出て落ちた奴を拾ってきて、学校で食う。菜園の西側が山城屋という質屋の庭続きで、この質屋に勘太郎という十三四の倅が居た。勘太郎は無論弱虫である。弱虫の癖に四つ目垣を乗りこえて、栗を盗みにくる。ある日の夕方折戸の蔭に隠れて、とうとう勘太郎を捕まえてやった。その時勘太郎は逃げ路を失って、一生懸命に飛びかかってきた。向うは二つばかり年上である。弱虫だが力は強い。鉢の開いた頭を、こっちの胸へ宛ててぐいぐい押した拍子に、勘太郎の頭がすべって、おれの袷の袖の中にはいった。邪魔になって手が使えぬから、無暗に手を振ったら、袖の中にある勘太郎の頭が、右左へぐらぐら靡いた。しまいに苦しがって袖の中から、おれの二の腕へ食い付いた。痛かったから勘太郎を垣根へ押しつけておいて、足搦をかけて向うへ倒してやった。山城屋の地面は菜園より六尺がた低い。勘太郎は四つ目垣を半分崩して、自分の領分へ真逆様に落ちて、ぐうと云った。勘太郎が落ちるときに、おれの袷の片袖がもげて、急に手が自由になった。その晩母が山城屋に詫びに行ったついでに袷の片袖も取り返して来た。\nこの外いたずらは大分やった。大工の兼公と肴

### 分かち書きにする

In [31]:
# 分かち書き用の関数定義
def wakati_text(text, pos=["名詞", "動詞"]):
    tokenizer = Tokenizer()
    doc = tokenizer.tokenize(text)
    wakati = None
    word_list = []
    for token in doc:
        p = token.part_of_speech.split(",")[0]
        if p in pos:
            word_list.append(token.base_form)
    if 0 < len(word_list):
        wakati = " ".join(word_list)
    return wakati

12個のテキストを分かち書きするのにすごく時間がかかるので，最初の1000文字だけ使います。

In [46]:
wakati_list = []
for clean_text in clean_text_list:
    wakati_list.append(wakati_text(clean_text[0:3000], ["名詞", "動詞"])) #ここと学習用の部分の文字数を変更（3000の部分）したり、名詞と動詞以外のやつを分析にいれてみたりする。

In [47]:
# 最初のテキストデータの最初の1000文字を表示
wakati_list[0][0:3000]

'一 親譲り 無鉄砲 供 時 損 する いる 小学校 居る 時分 学校 二 階 飛び降りる 一 週間 腰 抜かす 事 ある 無闇 する 聞く 人 ある 知れる 理由 新築 二 階 首 出す いる 同級生 一 人 冗談 威張る そこ 飛び降りる 事 出来る 弱虫 ー い 囃す 小使 負ぶさる 帰る 来る 時 おやじ 眼 する 二 階 飛び降りる 腰 抜かす 奴 ある 云う 次 抜かす 飛ぶ 見せる 答える 親類 もの 西洋 製 ナイフ 貰う 奇麗 刃 日 翳す 友達 見せる いる 一 人 光る 事 光る 切れる そう 云う 切れる 事 ある 何 切る みせる 受け合う そん 君 指 切る みる 注文 する 指 通り 右 手 親指 甲 はする 切り込む 幸 ナイフ の 親指 骨 今 親指 手 付く いる 創痕 死ぬ 消える 庭 東 二 十 歩 行く 尽す 南 上がり 菜園 ある 真中 栗 木 一 本 立つ いる これ 命 大事 栗 実 熟 する 時分 起き抜け 背戸 出る 落ちる 奴 拾う くる 学校 食う 菜園 西側 山城 屋 質屋 庭 続き 質屋 勘太郎 十 三 四 倅 居る 勘太郎 弱虫 弱虫 癖 四つ 目 垣 乗りこえる 栗 盗む くる 日 夕方 折戸 蔭 隠れる 勘太郎 捕まえる やる 時 勘太郎 逃げる 路 失う 一生懸命 飛びかかる くる 向う 二つ 年上 弱虫 力 鉢 開く 頭 こっち 胸 宛てる 押す 拍子 勘太郎 頭 すべる おれ 袷 袖 中 はいる 邪魔 なる 手 使える 手 振る 袖 中 ある 勘太郎 頭 右左 靡く がる 袖 中 おれ 二の腕 食う 付く 勘太郎 垣根 押しつける おく 足 搦 かける 向う 倒す やる 山城 屋 地面 菜園 六 尺 がた 勘太郎 四つ 目 垣 半分 崩す 自分 領分 真逆様 落ちる ぐうと 云う 勘太郎 落ちる とき おれ 袷 袖 もげる 急 手 自由 なる 晩 母 山城 屋 詫び 行く ついで 袷 袖 取り返す 来る 外 いたずら やる 大工 兼 公 肴 屋 角 つれる 茂作 人参 畠 あらす 事 ある 人参 芽 出揃う 処 藁 一 面 敷く ある 上 三 人 半日 相撲 とる つづける 取る 人参 みんな 踏みつぶす れる しまう 古川 持つ いる 田圃 井戸 埋める 尻 持ち込む れる 事 ある 孟宗 節

### BoWの計算

In [48]:
vectorizer = CountVectorizer()
vectorizer.fit(wakati_list)
X = vectorizer.transform(wakati_list).toarray()

### テキストごとのクラスを用意
1：夏目漱石 2：芥川竜之介　3:太宰治

In [49]:
y = np.array([1, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3])

### 単純ベイズ分類器の学習

In [50]:
clf = GaussianNB()

In [51]:
clf.fit(X, y)

GaussianNB()

In [52]:
clf.score(X, y)

1.0

### テスト用文書の読み込み

In [53]:
test_file_list = ["waganeko_content.txt", "kumo_content.txt", "melos_content.txt"]
test_text_list = []
for file in test_file_list:
    test_text_list.append(file_load(file))

### クレンジング

In [54]:
test_clean_list = []
for text in test_text_list:
  test_clean_list.append(cleansing(text))

### テスト用文書の分かち書き

最初の1000文字だけ使っています。

In [55]:
test_wakati_list = []
for clean_text in test_clean_list:
  test_wakati_list.append(wakati_text(clean_text[0:1000], ["名詞", "動詞"]))

In [56]:
test_wakati_list[0][0:3000] #ここの数値を変更したりする。

'一 吾輩 猫 名前 どこ 生れる 見当 つく 何 する 所 ニャーニャー 泣く いた事 記憶 する いる 吾輩 ここ 始める 人間 もの 見る あと 聞く それ 書生 人間 中 一番 獰悪 種族 そう 書生 の 我々 捕える 煮る 食う 話 当時 何 考 思う 彼 掌 載せる られる スー 持ち上げる られる 時 する 感じ ある 掌 上 落ちつく 書生 顔 見る の 人間 もの 見る 始 時 妙 もの 思う 感じ 今 残る いる 一 毛 装飾 する れる はず 顔 する 薬缶 その後 猫 逢う 輪 一 度 出会う する 事 なる 顔 真中 突起 する いる 穴 中 ぷうぷうと 煙 吹く 咽 する 弱る これ 人間 飲む 煙草 もの 事 頃 知る 書生 掌 裏 心持 坐る おる する 非常 速力 運転 する 始める 書生 動く の 自分 動く の 分る 眼 廻る 胸 なる 助かる 思う いる さる 音 する 眼 火 出る それ 記憶 する いる あと 何 事 考え出す 分る 気 付く 見る 書生 いる たくさん おる 兄弟 一疋 見える 肝心 母親 姿 隠す しまう 上今 所 違う 眼 明く いる られる 何 容子 這い出す 見る 非常 吾輩 藁 上 急 笹原 中 棄てる られる の 思い 笹原 這い出す 向う 池 ある 吾輩 池 前 坐る する 考える 見る これ 分別 出る する 泣く 書生 迎 来る くれる 考え付く ニャー ニャー 試み やる 見る 誰 来る 池 上 風 渡る 日 暮れ かかる 腹 非常 減る 来る 泣く 声 出る 仕方 何 食物 ある 所 ある 決心 する そる 池 左 廻る 始める 非常 そこ 我慢 する 無理やり 這う 行く 事'

### BoWの計算

In [57]:
T = vectorizer.transform(test_wakati_list).toarray()

### 学習済の分類器を用いて推定

In [58]:
clf.predict(T)#今回のテストでは

array([1, 3, 1])